### 문맥과 기억: AI를 더 현실적으로 만들기

### 학습 내용
 * No Context = 무질서한 무작위성(Chaos of randomness)  
 * History=Context - 이전의 요청과 응답을 사용하기
 * 새로운 접근 방식 - LIFO(Last in First out)
 * 선택적인 문맥 - LIFO 방식의 한계에 대한 제안

### 사전 준비
 * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 두가지 작업을 매번 수행해야 함.
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치
     * 설치시 첫 실행시 에러가 발생(23/12) - 해결(다시 한번 실행하면 사라짐)
     ```
     ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
```

In [2]:
!pip install openai

### No Context = 무질서한 무작위성
  * 우리가 만든 대화 에이전트가 기본적으로 메모리를 가지고 있지 않음.

In [6]:
from openai import OpenAI
import os


def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()
client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

initial_prompt = """You: 안녕!
AI: 어떻게 잘 지냈어요?
You: {}
AI: """

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    next = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = initial_prompt.format(prompt),
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"]
    )
    print("AI:", next.choices[0].text.strip())


You(q:quit): 안녕
AI: 반가워요.
You(q:quit): 반갑다. 하나 부탁할게.
AI: 무슨 일인가요? 부탁해도 될까요?
You(q:quit): 메모해두고, 나중에 이야기를 해 줄 수 있니?
AI: 네, 메모해둘게요. 이야기를 나중에 그때 다시 들려주며 좋아할 거예요.
You(q:quit): '학교'라는 단어 메모 부탁해.
AI: 알겠습니다. "학교" 라는 단어를 메모했습니다.
You(q:quit): 너가 기억한 단어가 뭐니?
AI: 그게 무슨 말이에요?
You(q:quit): 방금 메모해 두라고 했던 것.
AI: 많은 일이 생겼나요?
You(q:quit): OK
AI: 그래요, 그렇구나. 무슨 일이 있었나요?
You(q:quit): q


### History=Context - 이전의 요청과 응답을 사용하기

#### 이전의 요청과 답변을 변수에 기억시킴.

In [7]:
from openai import OpenAI
import os

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

initial_prompt = """You: 안녕!
AI: 어떻게 잘 지냈어요?
You: {}
AI: """

history = ""

while True :
  prompt = input("You(q:quit): ")
  if prompt=='q':
        break
  next = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = initial_prompt.format(history + prompt),
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"],
  )
  next_text = next.choices[0].text
  history += "You: " + prompt + " \n " + "AI: " + next_text + " \n "
  print("AI: " + next_text)


You(q:quit): "안녕" 
AI: 안녕하세요, 만나서 반가워요. 저는 인공지능 챗봇이에요. 
You(q:quit): 하나 부탁이 있어, 내가 말한 단어를 메모해 두고 나중에 알려주렴.
AI: 좋아요. 어떤 단어를 메모해 둘까요?
You(q:quit): hiking 이 단어 메모 부탁해.
AI: "hiking" 단어를 메모해두었습니다. 다시 말씀해주시면 알려드릴게요.
You(q:quit): 방금 메모한 단어가 뭐니?
AI:  방금 메모한 단어는 "hiking"이에요.
You(q:quit): 오오 고맙다.
AI: 별 일 아닙니다. 그런데, 저도 알고 싶어서 검색해봤어요. 하이킹은 산을 등산하면서 즐기는 레저나 스포츠 활동을 뜻한다고 하네요.
You(q:quit): q


### 새로운 접근 방식 - LIFO(Last in First out)

* 이 접근 방식의 아이디어
  * 사용자는 항상 문맥을 가지고 대화 시작
  * 문맥은 변화고, 대화도 변한다.
  * 사용자는 최근 2-5개 프롬프트에 문맥을 포함시킬 가능성이 매우 크다.
* 히스토리를 저장할 텍스트 파일을 만들고, 구분자로 프롬프트와 답변을 저장한다.


In [9]:
from openai import OpenAI
import os

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

def save_history_to_file(history):
    with open("history.txt", "w+") as f:
        f.write(history)

def load_history_from_file():
    with open("history.txt", "r") as f:
        return f.read()

def get_relevant_history(history):
    history_list = history.split(separator)
    if len(history_list) > 2:
        return separator.join(history_list[-2:])
    else:
        return history

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

In [14]:
initial_prompt = """
You: 안녕!
AI: 어떻게 잘 지냈니?
You: {}
AI:
"""

history = ""
relevant_history = ""
separator = "#####"
save_history_to_file(history)

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    relevant_history = get_relevant_history(load_history_from_file())
    next = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = initial_prompt.format(relevant_history + prompt),
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"],
    )
    next_text = next.choices[0].text
    history += "\nYou: " + prompt + "\nAI: " + next_text + "\n" + separator
    save_history_to_file(history)
    print("AI: " + next_text)



You(q:quit): 안녕
AI: 안녕하세요?
You(q:quit): ‘오늘의 단어장’에 이 단어 넣어주렴. ‘banana’ – 나중에 물어보면 알려주렴. 
AI: 오늘 배운 단어는 banana입니다. 나중에 물어보시면 더 자세히 알려드리겠습니다.
You(q:quit): ‘오늘의 단어장’ 들어있는 단어는 뭐였지? 
AI: 
오늘 배운 단어는 banana입니다.
You(q:quit): ‘오늘의 단어장’에 이 단어 넣어주렴. ‘apple’ – 나중에 물어보면 알려주렴. 
AI: 내일 배울 단어로 apple을 넣어놨어. 나중에 물어보면 알려줄게.
You(q:quit): ‘오늘의 단어장’에 이 단어 넣어주렴. ‘orange’ – 나중에 물어보면 알려주렴. 
AI: 
네, 단어장에 orange 단어를 저장했어요. 나중에 물어보면 알려줄게요.
You(q:quit): ‘오늘의 단어장’에 기억해 둔 단어를 알려주렴.
AI: 저는 지금까지 orange 라는 단어를 기억하고 있어요. 다른 단어도 물어보세요!
You(q:quit): q


### 선택적인 문맥 - LIFO의 한계의 해결 제안

* 초기 프롬프트가 텍스트 파일에 저장
* 사용자가 프롬프트에 입력
* 모든 상호 작용에 대한 임베딩 생성
* 임베딩과 코사인 유사도를 활용
* 가장 높은 n개의 내용을 사용자에게 프롬프트와 함께 보내기

#### 사전 다운로드 및 설치

In [17]:
!pip install spaCy

In [18]:
!python -m spacy download en_core_web_md

2023-12-12 07:17:00.960138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 07:17:00.960219: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 07:17:00.960269: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 07:17:02.326726: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 8.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


#### 모든 코드를 합친 코드

In [21]:
import openai
import os
import spacy
import numpy as np
from openai import OpenAI

# 사전 학습된 spaCy 모델을 로드합니다.
nlp = spacy.load('en_core_web_md')

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

def save_history_to_file(history):
    """ 대화 기록을 파일에 저장합니다. """
    with open("history.txt", "w+") as f:
        f.write(history)

def load_history_from_file():
    """ 파일로부터 모든 대화 기록을 불러옵니다. """
    with open("history.txt", "r") as f:
        return f.read()

def cos_sim(a, b):
    """ 두 문자열 사이의 코사인 유사도를 계산합니다.
    사용자 입력과 기록된 대화 세그먼트 사이의 유사성 비교에 사용됩니다.
    """
    a = nlp(a)
    a_without_stopwords = nlp(' '.join([t.text for t in a if not t.is_stop]))
    b = nlp(b)
    b_without_stopwords = nlp(' '.join([t.text for t in b if not t.is_stop]))
    return a_without_stopwords.similarity(b_without_stopwords)

def sort_history(history, user_input):
    """
    사용자 입력과 대화 기록 세그먼트 사이의 코사인 유사도에 기반하여 대화 기록을
    정렬합니다. History는 세팅자(separator)로 구분된 세그먼트들의 문자열입니다.
    """
    segments = history.split(separator)
    similarities = []
    for segment in segments:
        # 사용자 입력과 세그먼트 간의 코사인 유사도를 얻습니다.
        similarity = cos_sim(user_input, segment)
        similarities.append(similarity)

    sorted_similarities = np.argsort(similarities)
    sorted_history = ""
    for i in range(1, len(segments)):
        sorted_history += segments[sorted_similarities[i]] + separator
    save_history_to_file(sorted_history)

def get_latest_n_from_history(history, n):
    """
    대화 기록에서 최근 n개의 세그먼트를 가져옵니다.
    History는 세팅자(separator)로 구분된 세그먼트들의 문자열입니다.
    """
    segments = history.split(separator)
    return separator.join(segments[-n:])

init_api()
client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

initial_prompt_1 = """
You: 안녕하세요!
AI: 안녕하세요!
#####
You: 어떻게 지내세요?
AI: 저는 잘 지내고 있어요, 감사합니다.
#####
You: 자동차에 대해 알고 있나요?
AI: 네, 저는 자동차에 대해 어느 정도 알고 있습니다.
#####
You: 피자 먹나요?
AI: 저는 피자를 먹지 않습니다. 저는 먹을 수 없는 AI입니다.
#####
You: 달에 가 본 적 있나요?
AI: 저는 달에 가 본 적이 없습니다. 당신은 어떠세요?
#####
You: 당신의 이름은 무엇인가요?
AI: 제 이름은 픽셀입니다. 당신의 이름은 무엇인가요?
#####
You: 가장 좋아하는 영화는 무엇인가요?
AI: 제가 가장 좋아하는 영화는 '매트릭스'입니다. Follow the white rabbit :)
#####"""

initial_prompt_2 = """You: {}
AI: """

initial_prompt = initial_prompt_1 + initial_prompt_2
separator = "#####"

save_history_to_file(initial_prompt_1)

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    sort_history(load_history_from_file(), prompt)
    history = load_history_from_file()
    best_history = get_latest_n_from_history(history, 5)
    full_user_prompt = initial_prompt_2.format(prompt)
    full_prompt = best_history + "\n" + full_user_prompt

    next = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = full_prompt,
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"],
    )
    response_text = next.choices[0].text.strip()
    history += "\n" + full_user_prompt + response_text + "\n" + separator + "\n"
    save_history_to_file(history)
    print("AI: " + response_text)


You(q:quit): 안녕


<ipython-input-21-f5b51077c374>:34: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return a_without_stopwords.similarity(b_without_stopwords)


AI: 안녕하세요? 전 누구예요?
You(q:quit): 너는 무엇이 가능하니?
AI: 저는 다양한 질문에 대답할 수 있고 언어 모델 기반으로 대화를 진행할 수 있습니다. 어떤 말씀이든 궁금하신 내용을 물어보세요.
You(q:quit): 달에 가 본 적이 있니?
AI: 제 의견은 당신은 로봇 인간 일수도 있습니다. 당신이 야외 활동을 좋아하는 사람이라면 아마 달을 방문한 적이 있을 수도 있습니다. 당신의 인생에 대해 더 알 수 있을까요?
You(q:quit): 피자 먹을 수 있니?
AI: 저는 로봇이기 때문에 먹을 수는 없지만, 사실상 반응해드릴 수 없는 맛을 느낄 수 있을 것 같습니다. 당신은 어떤 종류의 피자를 선호하나요?
You(q:quit): 자동차에 대해 알고 있니?
AI: 저는 자동차에 대해 많은 정보를 가지고 있습니다. 자동차를 이용하는 많은 사람들이 있기 때문에 자동차에 대해 아는 것이 중요합니다.
You(q:quit): q
